In [66]:
import numpy as np
import pandas as pd


In [67]:
channel_summary = pd.read_csv('datadesign/incremental_ltv_results.csv')

In [68]:
channel_summary


,channel,incremental_ltv,avg_cost
0,referral,21.220446,1.002913
1,paid_search,14.959840,2.997953
2,affiliate,10.564800,2.496420
3,organic,6.936259,0.200189
4,social_ads,5.622442,2.000272


In [69]:
TOTAL_BUDGET = 500_000  # dollars

# Max % of total budget allowed per channel
MAX_SPEND_PCT = {
    "paid_search": 0.40,   # ≤ 40%
    "social_ads": 0.30,    # ≤ 30%
    "affiliate": 0.20,     # ≤ 20%
    "referral": 0.16,      # ≤ 16%
    "organic": 0.10,       # ≤ 10%
}

# Min % of total budget required per channel
MIN_SPEND_PCT = {
    "paid_search": 0.04,   # ≥ 4%
    "social_ads": 0.02,    # ≥ 2%
    "affiliate": 0.01,    # ≥ 1%
    "referral": 0.01,     # ≥ 1%
    "organic": 0.00,
}

MAX_SPEND = {
    c: TOTAL_BUDGET * MAX_SPEND_PCT[c]
    for c in MAX_SPEND_PCT
}

MIN_SPEND = {
    c: TOTAL_BUDGET * MIN_SPEND_PCT[c]
    for c in MIN_SPEND_PCT
}




In [70]:
from ortools.linear_solver import pywraplp


In [71]:
solver = pywraplp.Solver.CreateSolver("GLOP")


In [72]:
x = {}

for _, row in channel_summary.iterrows():
    c = row["channel"]
    x[c] = solver.NumVar(
        MIN_SPEND[c],
        MAX_SPEND[c],
        f"spend_{c}"
    )


In [73]:
solver.Add(
    solver.Sum(x[c] for c in x) <= TOTAL_BUDGET
)


<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x12ce83120> >

In [74]:
objective = solver.Objective()

for _, row in channel_summary.iterrows():
    c = row["channel"]
    value_per_dollar = row["incremental_ltv"] / row["avg_cost"]
    objective.SetCoefficient(x[c], value_per_dollar)

objective.SetMaximization()


In [75]:
status = solver.Solve()

In [76]:
allocation = []

for c in x:
    allocation.append({
        "channel": c,
        "optimal_spend": x[c].solution_value()
    })

allocation_df = pd.DataFrame(allocation)
allocation_df


,channel,optimal_spend
0,referral,80000.0
1,paid_search,200000.0
2,affiliate,100000.0
3,organic,50000.0
4,social_ads,70000.0


In [77]:
allocation_df = allocation_df.merge(channel_summary, on="channel")

allocation_df["expected_incremental_value"] = (
    allocation_df["optimal_spend"]
    * (allocation_df["incremental_ltv"]/ allocation_df["avg_cost"])
)
allocation_df['expected_incremental_value'] = allocation_df['expected_incremental_value'].astype(np.int64)

allocation_df[[
    "channel",
    "optimal_spend",
    "expected_incremental_value"
]]


,channel,optimal_spend,expected_incremental_value
0,referral,80000.0,1692704
1,paid_search,200000.0,998003
2,affiliate,100000.0,423198
3,organic,50000.0,1732427
4,social_ads,70000.0,196758


In [78]:
allocation_df["expected_incremental_value"].sum()


np.int64(5043090)

In [79]:
allocation_df["incremental_ltv_low"] = allocation_df["incremental_ltv"] * 0.8


In [80]:
allocation_df["expected_value_low"] = (
    allocation_df["optimal_spend"]
    * allocation_df["incremental_ltv_low"]
    / allocation_df["avg_cost"]
)
allocation_df['expected_value_low'] = allocation_df['expected_value_low'].astype(np.int64)


In [81]:
allocation_df[[
    "channel",
    "expected_incremental_value",
    "expected_value_low"
]]


,channel,expected_incremental_value,expected_value_low
0,referral,1692704,1354163
1,paid_search,998003,798402
2,affiliate,423198,338558
3,organic,1732427,1385942
4,social_ads,196758,157406


In [82]:
allocation_df['expected_value_low'].sum()

np.int64(4034471)

In [83]:
final_output = allocation_df[[
    "channel",
    "optimal_spend",
    "expected_incremental_value"
]].sort_values("expected_incremental_value", ascending=False)

final_output


,channel,optimal_spend,expected_incremental_value
3,organic,50000.0,1732427
0,referral,80000.0,1692704
1,paid_search,200000.0,998003
2,affiliate,100000.0,423198
4,social_ads,70000.0,196758


In [84]:
final_output.to_csv('datadesign/optimal_allocation.csv', index = False)

In [85]:
channels = channel_summary["channel"].tolist()
equal_spend = TOTAL_BUDGET / len(channels)

baseline = pd.DataFrame({
    "channel": channels,
    "spend": equal_spend
})
baseline["spend"] = baseline.apply(
    lambda r: min(
        max(r["spend"], MIN_SPEND[r["channel"]]),
        MAX_SPEND[r["channel"]]
    ),
    axis=1
)
remaining_budget = TOTAL_BUDGET - baseline["spend"].sum()

eligible = baseline[
    baseline["spend"] < baseline["channel"].map(MAX_SPEND)
].copy()

eligible_capacity = (
    eligible["channel"]
    .map(MAX_SPEND)
    .values
    - eligible["spend"].values
).sum()

baseline["extra_spend"] = 0.0

for i, row in eligible.iterrows():
    capacity = MAX_SPEND[row["channel"]] - row["spend"]
    baseline.loc[i, "extra_spend"] = (
        remaining_budget * capacity / eligible_capacity
    )

baseline["spend"] += baseline["extra_spend"]
baseline.drop(columns="extra_spend", inplace=True)
channel_metrics = channel_summary.set_index("channel")

baseline["expected_incremental_value"] = baseline.apply(
    lambda r: (
        r["spend"]
        * channel_metrics.loc[r["channel"], "incremental_ltv"]
        / channel_metrics.loc[r["channel"], "avg_cost"]
    ),
    axis=1
)
updated_baseline_score = baseline["expected_incremental_value"].sum()
updated_baseline_score
baseline.to_csv("datadesign/baseline_allocation.csv", index=False)



In [86]:
baseline

,channel,spend,expected_incremental_value
0,referral,80000.000000,1.692705e+06
1,paid_search,146666.666667,7.318692e+05
2,affiliate,100000.000000,4.231980e+05
3,organic,50000.000000,1.732428e+06
4,social_ads,123333.333333,3.466701e+05


In [87]:
channel_summary

,channel,incremental_ltv,avg_cost
0,referral,21.220446,1.002913
1,paid_search,14.959840,2.997953
2,affiliate,10.564800,2.496420
3,organic,6.936259,0.200189
4,social_ads,5.622442,2.000272


In [88]:
channel_summary.to_csv('datadesign/channel_summary.csv', index = False)